# Projeto 4 - Analise de crédito com IA

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import seaborn as sns

* ID: ID de cada cliente
* LIMIT_BAL: Valor do crédito concedido em NT dólares (inclui crédito individual e familiar/suplementar
* SEXO: Gênero (1=masculino, 2=feminino)
* EDUCAÇÃO: (1=pós-graduação, 2=universidade, 3=ensino médio, 4=outros, 5=desconhecido, 6=desconhecido)
* CASAMENTO: Estado civil (1=casado, 2=solteiro, 3=outros)
* IDADE: idade em anos
* PAY_0: Status do reembolso em setembro de 2005 (-1=pagamento devido, 1=atraso no pagamento por um mês, 2=atraso no pagamento por dois meses, … 8=atraso no pagamento por oito meses, 9=atraso no pagamento por nove meses e acima)
* PAY_2: Status do reembolso em agosto de 2005 (escala igual à acima)
* PAY_3: Status do reembolso em julho de 2005 (escala igual à acima)
* PAY_4: Status do reembolso em junho de 2005 (escala igual à acima)
* PAY_5: Status do reembolso em maio de 2005 (escala igual à acima)
* PAY_6: Status do reembolso em abril de 2005 (escala igual à acima)
* BILL_AMT1: Valor do extrato da conta em setembro de 2005 (NT dólar)
* BILL_AMT2: Valor do extrato da conta em agosto de 2005 (NT dólar)
* BILL_AMT3: Valor do extrato da conta em julho de 2005 (NT dólar)
* BILL_AMT4: Valor do extrato da conta em junho de 2005 (NT dólar)
* BILL_AMT5: Valor do extrato da conta em maio de 2005 (NT dólar)
* BILL_AMT6: Valor do extrato da conta em abril de 2005 (NT dólar)
* PAY_AMT1: Valor do pagamento anterior em setembro de 2005 (NT dólares)
* PAY_AMT2: Valor do pagamento anterior em agosto de 2005 (NT dólar)
* PAY_AMT3: Valor do pagamento anterior em julho de 2005 (NT dólar)
* PAY_AMT4: Valor do pagamento anterior em junho de 2005 (NT dólar)
* PAY_AMT5: Valor do pagamento anterior em maio de 2005 (NT dólar)
* PAY_AMT6: Valor do pagamento anterior em abril de 2005 (NT dólar)
* default.payment.next.month: Pagamento padrão (1=sim, 0=não)

In [ ]:
dados_bancarios = pd.read_csv('dados_cartao_credito.csv')
dados_bancarios

In [ ]:
dados_bancarios = dados_bancarios.drop('ID', axis = 1) 

# Dummies

* Como vamos trabalhar com dados de classificação em uma regressão, precisamos pegar dummies pros dados categóricos.
* Assim, conseguimos inserir um beta que vai multiplicar o valor da dummie e adicionar um valor a nossa equação caso a pessoa em questão se enquadre em alguma classificação.
* Com dummies nós conseguimos medir exatamente a sensibilidade de adicionar uma variável ou não.


Y = A + B DF + B * DM + B * DF

Y = 1250 + 250 * 1 + 750 * 1 + 1000 * 1

EF = 1500
EM = 2250
EFA = 3250

In [ ]:
dicionario_colunas_dummies = {
            'EDUCATION': 'EDU',
            'MARRIAGE': 'MAR'
        }

In [ ]:
for column, prefix in dicionario_colunas_dummies.items():
    dummies = pd.get_dummies(dados_bancarios[column], prefix=prefix)
    dados_bancarios = pd.concat([dados_bancarios, dummies], axis=1)
    dados_bancarios = dados_bancarios.drop(column, axis=1)
    
dados_bancarios

In [ ]:
#pegando x e Y

Y = dados_bancarios['default.payment.next.month'].copy()
X = dados_bancarios.drop('default.payment.next.month', axis=1).copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)

In [ ]:
escalador_treino = StandardScaler()
escalador_teste = StandardScaler()


X_train = pd.DataFrame(escalador_treino.fit_transform(X_train), columns=X.columns) #só precisamos escalar o X pq o Y é 0 ou 1. 
X_test = pd.DataFrame(escalador_teste.fit_transform(X_test), columns=X.columns) 

X_train

### Treinando o modelo

In [ ]:
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

### Fazendo previsões de default.

* Em x% dos casos ele disse que ia dar default e realmente deu e ele disse que não ia dar e realmente não deu.

In [ ]:
"Acertos: {:.2f}%".format(modelo.score(X_test, y_test) * 100)

### Isso é a mesma coisa que fazer isso:

In [ ]:
prevendo_defaults = modelo.predict(X_test)

prevendo_defaults

In [ ]:
from sklearn.metrics import accuracy_score

acertos = accuracy_score(y_test.values, prevendo_defaults)
acertos

### Matriz de confusão

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test.values, prevendo_defaults)
cm

In [ ]:
#0/0 - 0/1
#1/0 - 1/1

# Análise

* Qual erro é pior pra você?
* Otimizar o modelo.
* ML é uma otimização de processo: o algoritmo é melhor do que existe hoje? Ele erra menos ou mais?
* No caso dos bancos, o erro tipo 1 é crucial. Deixar de dar crédito pra alguém que ia pagar é menos pior do que tomar default.
* Deixamos de ganhar dinheiro em 1040 casos e perdemos em 143. Calcular o custo disso tudo e ver se isso é melhor do que está sendo praticado hoje.